<a href="https://colab.research.google.com/github/Gregory-Eales/RL-TicTacToe/blob/master/RL_TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependencies

In [3]:
!pip3 install dropbox

    100% |████████████████████████████████| 512kB 24.8MB/s 


In [4]:
import numpy as np
import keras
import dropbox
from keras.layers import Convolution2D as Conv2D
from keras.layers import Dense
from keras.layers import Flatten

Using TensorFlow backend.


Defining Training Enviroment

In [0]:
class TrainENV(object):
  
  def __init__(self, num_models=2):
    self.num_models = 2
    self.board = np.zeros([3, 3])
    self.dbx_token = "qCE45upb8QAAAAAAAAAAEkIQWUdP5SqMqWG9I5C_3DDAN_Vlleeyjvc5Sv4vyWp9"
    self.dbx = dropbox.Dropbox(self.dbx_token)
    self.initialize_models()
    self.build_model_architecture()
    
  ###############################
  # DEEP NEURAL NETWORK METHODS #
  ###############################
  
  # defines and stores models in model dict
  def initialize_models(self):
    self.model = {}
    for i in range(self.num_models):
      self.model["M" + str(i+1)] = keras.models.Sequential()
    
    
  # builds initialized model with defined architecture
  def build_model_architecture(self):
    for i in range(self.num_models):
      self.model["M" + str(i+1)].add(Conv2D(64, kernel_size=2, activation='relu', padding="same", input_shape=(3,3,1)))
      self.model["M" + str(i+1)].add(Conv2D(32, kernel_size=2, padding="same", activation='relu'))
      self.model["M" + str(i+1)].add(Flatten())
      self.model["M" + str(i+1)].add(Dense(32, activation='softmax'))
      self.model["M" + str(i+1)].add(Dense(9, activation='softmax'))
      self.model['M' + str(i+1)].compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
  
  # saves models in project folder
  def save_models(self):
    for i in range(self.num_models):
     # self.model["M"+str(i+1)].
     pass
  
  # loads models from project folder
  def load_models(self):
    pass
  
  ###########################
  # GAME ENVIROMENT METHODS #
  ###########################
  
  # runs the main game loop
  def play_human(self):
    turn = "human"
    # reset board
    self.board = np.zeros([3, 3])
    # get player move
    
    # check if move is valid
    
    # if valid change turn
    
    pass
  
  def rand_train_ai(self, model="M1", epochs=100):
    
    for i in range(100):
      playing = True
      while playing:
        turn = "ai"
  
  def get_ai_move(self, model="M1"):
    
  
  
  
  # checks to see if move is valid
  def check_move(self):
    pass
  
  # gets human move
  def get_move(self):
    pass
  
  # gets ai move from model
  def get_model_move(self):
    pass
  
  # check to see if there is a winner
  def check_winner(self):
    
    # check horizontal
    for i in range(3):
      
      if self.board[i][0] == self.board[i][1] and self.board[i][1] == self.board[i][2]:
        if self.board[i][0] != 0:
          return self.board[i][0]
        
      if self.board[0][i] == self.board[1][i] and self.board[1][i] == self.board[2][i]:
        if self.board[0][i] != 0:
          return self.board[0][i]
      
    if self.board[0][0] == self.board[1][1] and self.board[1][1] == self.board[2][2]:
      if self.board[0][0] != 0: 
        return self.board[0][0]
    
    if self.board[0][2] == self.board[1][1] and self.board[1][1] == self.board[2][0]:
      if self.board[1][1] != 0:
        return self.board[1][1]
    
    
    return None
   
      
  
  # prints board state
  def print_board(self):
    board = []
    
    for i in range(3):
      row = []
      for j in range(3):
        
        if self.board[i][j] == 0:
          row.append(" ")
          
        elif self.board[i][j] == 1:
          row.append("X")
        
        elif self.board[i][j] == -1:
          row.append("O")
        
      board.append(row)
    
    print("-------------")
    print("|", board[0][0], "|", board[0][1], "|", board[0][2], "|")
    print("-------------")
    print("|", board[1][0], "|", board[1][1], "|", board[1][2], "|")
    print("-------------")
    print("|", board[2][0], "|", board[2][1], "|", board[2][2], "|")
    print("-------------")
    
  
  
  ####################
  # DROP BOX METHODS #
  ####################
  
  # returns the current dropbox account
  def get_dbx_account(self):
    return(self.dbx.users_get_current_account())
  
  # uploads project file
  def upload_file_dbx(self):
    dbx.files_upload("RL-TicTacToe", '/RL-TicTacToe/')
  
  # downloads project file
  def download_file_dbx(self):
    pass

Testing ENV Elements

In [40]:
x = np.array([[1, 1, 0], [-1, -1, 1], [1, 1, -1]]).reshape(1, 3, 3, 1)
y = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0]).reshape(1, 9)

env = TrainENV()
env.get_dbx_account()

#env.model["M1"].fit(x, y, epochs=100)


env.board = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])


env.print_board()



-------------
| X |   |   |
-------------
|   | O |   |
-------------
|   |   | X |
-------------
